# DEPRECATED! GO HERE FOR LATEST WORKING: <a href="../../job/space_track/filter_raw_data_payload_maneuvers.ipynb">job/space_track/filter_raw_data_payload_maneuvers.ipynb</a>

#### Please leave this alone though!

In [1]:
raise Exception("STOP! THIS IS DEPRECATED!")

Exception: STOP! THIS IS DEPRECATED!

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
csv_store_path = "../../../siads591 data/space_track_raw/csv/"
filtered_store_path = "../../../siads591 data/filtered_raw/"
output_file_name = "payload.pkl"

In [ ]:
# setting up default and initial variables
columns = ['NORAD_CAT_ID', 'MEAN_MOTION', 'ECCENTRICITY', 'SEMIMAJOR_AXIS', 'INCLINATION', 'OBJECT_TYPE', 'EPOCH']
dtypes = {
    'NORAD_CAT_ID':np.uint16,
    'SEMIMAJOR_AXIS':np.float64,
    'INCLINATION':np.float64,
    'MEAN_MOTION':np.float64,
    'ECCENTRICITY':np.float64,
}

In [ ]:
dfs = []
files = sorted([x for x in os.listdir(f'{csv_store_path}/') if x.endswith(".csv.gz")])
for f in tqdm(files):
    if f.endswith(".csv.gz"):
#        LEO = Mean Motion > 11.25 and Eccentricity < 0.25
        df = pd.read_csv(f'{csv_store_path}{f}', usecols=columns, parse_dates=['EPOCH'], infer_datetime_format=True, index_col='EPOCH', dtype=dtypes, compression='gzip')
        df = df[(df.MEAN_MOTION > 11.25) & (df.ECCENTRICITY < 0.25) & (df.OBJECT_TYPE == "PAYLOAD")][['NORAD_CAT_ID', 'SEMIMAJOR_AXIS', 'INCLINATION']]
        df['SEMIMAJOR_AXIS'] = (df.SEMIMAJOR_AXIS * 1000).astype(np.uint32)
        df['INCLINATION'] = (df.INCLINATION * 10000).astype(np.uint32)
        dfs.append(df)
df = pd.concat(dfs)
df = df.rename(columns = {'SEMIMAJOR_AXIS':'SEMIMAJOR_AXIS_x1000', 'INCLINATION':'INCLINATION_x10000'})
df.to_pickle(f"{filtered_store_path}{output_file_name}.gz",compression="gzip")
print(f"Total number of rows {len(df)}")